In [24]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

In [25]:
#feature data for each node
x = torch.tensor([[1.2, 1.3, 1.1], [2.1, 2.2, 2.0], [3.5, 3.6, 3.4]], dtype=torch.float)

edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
y = torch.tensor([[1.4], [2.3], [3.7]], dtype=torch.float) #predicted variable

data = Data(x=x, edge_index=edge_index,y=y)

In [26]:
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        #for message passing and aggregation
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x) #apply ReLU activation
        
        x = self.conv2(x, edge_index)
        
        return x

In [27]:
#input channels - features per node
#output channels - number of features to predict
model = GNN(in_channels=3, hidden_channels=16, out_channels=1)

In [28]:
import torch.optim as optim
criterion = F.mse_loss 
optimizer = optim.Adam(model.parameters(), lr=0.05)

In [29]:
for epoch in range(200):
    optimizer.zero_grad() #reset previous gradients
    
    out = model(data) #forward: get predictions
    loss = criterion(out,data.y)
    
    loss.backward() #recalculate gradients
    optimizer.step() #adjust model parameters

    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch+1:03d}, Loss: {loss:.4f}')

Epoch: 010, Loss: 0.7109
Epoch: 020, Loss: 0.5815
Epoch: 030, Loss: 0.5311
Epoch: 040, Loss: 0.4933
Epoch: 050, Loss: 0.4481
Epoch: 060, Loss: 0.3880
Epoch: 070, Loss: 0.3174
Epoch: 080, Loss: 0.2367
Epoch: 090, Loss: 0.1574
Epoch: 100, Loss: 0.0967
Epoch: 110, Loss: 0.0653
Epoch: 120, Loss: 0.0554
Epoch: 130, Loss: 0.0516
Epoch: 140, Loss: 0.0470
Epoch: 150, Loss: 0.0428
Epoch: 160, Loss: 0.0388
Epoch: 170, Loss: 0.0351
Epoch: 180, Loss: 0.0316
Epoch: 190, Loss: 0.0284
Epoch: 200, Loss: 0.0430


In [30]:
model.eval() #disables Dropout

new_x = torch.tensor([[1.5, 1.6, 1.4], [2.5, 2.6, 2.4], [3.8, 3.9, 3.7]], dtype=torch.float)

new_data = Data(x=new_x, edge_index=edge_index)


In [31]:
with torch.no_grad(): #to save memory
    predictions = model(new_data)

print("Model Predictions:")
print(predictions)

Model Predictions:
tensor([[2.6908],
        [4.3161],
        [5.1187]])
